In [1]:
import pandas as pd

# Load the data, there are GN and GP datasets
file_name = "GN_Dataset_1.xlsx"
# Refer to the model.pdf for the target medicine according to the dataset
#target_medicine = "Trimethoprim/sulfamethoxazole".lower()
df = pd.read_excel(file_name)
df = df.drop(columns=["date_of_submission", "report_issued_date", "id", "type", "vitek_id"])

C:\Users\fin_t\AppData\Local\Temp\ipykernel_13404\2146399312.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.get_dummies(df, columns=["species", "submitted_sample", "bacteria_genus"])
for column in df.columns:
    if "S/I/R_" in str(column):
        df = pd.get_dummies(df, columns=[column])

In [3]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import numpy as np

In [4]:
# Choosing the features column and the target column
features_col = []
for col in df.columns:
    if "species" in col or "submitted_sample" in col or "bacteria_genus" in col or "S/I/R_" in col:
        features_col.append(col)

X = df[features_col]

allmeds = ["Trimethoprim/sulfamethoxazole","Cefovecin","Marbofloxacin","Amikacin","Amoxicillin/clavulanic acid","Imipenem","Enrofloxacin","Cefalexin","Gentamicin","Nitrofurantoin","Doxycycline"]
bestdepthsngn = [7,8,7,9,3,10,7,6,7,6,9]

ylist = []
for i in allmeds:
    ylist.append(df["ans_" + i.lower()])

modellist = []
total_y_test = []

for i in range(len(allmeds)):
    model = DecisionTreeClassifier(max_depth=bestdepthsngn[i],random_state=42, criterion="gini")
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = ylist[i].iloc[train_index], ylist[i].iloc[test_index]
        total_y_test = total_y_test + list(y_test)
        model.fit(X_train,y_train)
        modellist.append(modellist)


In [5]:
import joblib
for i in range(len(allmeds)):
    target_medicine = allmeds[i].replace("/","-")
    model_name = "model_" + target_medicine + "_gn.pkl"
    joblib.dump(modellist,model_name)

In [6]:
import pandas as pd

# Load the data, there are GN and GP datasets
file_name = "GP_Dataset_1.xlsx"
# Refer to the model.pdf for the target medicine according to the dataset
#target_medicine = "Trimethoprim/sulfamethoxazole".lower()
df = pd.read_excel(file_name)
df = df.drop(columns=["date_of_submission", "report_issued_date", "id", "type", "vitek_id"])

In [7]:
df = pd.get_dummies(df, columns=["species", "submitted_sample", "bacteria_genus"])
for column in df.columns:
    if "S/I/R_" in str(column):
        df = pd.get_dummies(df, columns=[column])


In [8]:
features_col = []
for col in df.columns:
    if "species" in col or "submitted_sample" in col or "bacteria_genus" in col or "S/I/R_" in col:
        features_col.append(col)

X = df[features_col]

allmeds = ["Trimethoprim/sulfamethoxazole","Cefovecin","Marbofloxacin","Amikacin","Amoxicillin/clavulanic acid","Enrofloxacin","Cefalexin","Nitrofurantoin","Doxycycline", "Vancomycin", "Clindamycin"]
bestdepthsngp = [6,6,4,10,7,3,9,6,4,4,5]

ylist = []
for i in allmeds:
    ylist.append(df["ans_" + i.lower()])

modellist = []
total_y_test = []

for i in range(len(allmeds)):
    model = DecisionTreeClassifier(max_depth=bestdepthsngn[i],random_state=42, criterion="gini")
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = ylist[i].iloc[train_index], ylist[i].iloc[test_index]
        total_y_test = total_y_test + list(y_test)
        model.fit(X_train,y_train)
        modellist.append(modellist)



In [9]:
for i in range(len(allmeds)):
    target_medicine = allmeds[i].replace("/","-")
    model_name = "model_" + target_medicine + "_gp.pkl"
    joblib.dump(modellist,model_name)